In [1]:
%pwd

'/home/farel/deep_learning/chicken-desease-classification/ressearch'

In [2]:
import os

os.chdir("../")

%pwd

'/home/farel/deep_learning/chicken-desease-classification'

In [3]:
# Initialize entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir:Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [4]:
# create the configuration manager
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_prepare_callback(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir), Path(config.tensorboard_root_log_dir)])
        
        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=os.path.join('', config.checkpoint_model_filepath)
        )
        
        return prepare_callbacks_config

In [5]:
# create the components
import os
import tensorflow as tf
import time


class PrepareCallback:
    def __init__(self, config:PrepareCallbacksConfig):
        self.config = config
        
        
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H:%M:%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir, f"tb_logs_at_{timestamp}")
        
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
        
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
        
    
    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callbacks, self._create_ckpt_callbacks]

2023-11-27 13:58:13.429266: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-27 13:58:13.488793: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 13:58:13.488842: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-27 13:58:13.488868: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-27 13:58:13.497969: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-27 13:58:13.498802: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [6]:
# test
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e    

[2023-11-27 13:58:15,851]: INFO: common: yaml file: config/configuration.yaml loaded successfully:]
[2023-11-27 13:58:15,855]: INFO: common: yaml file: params.yaml loaded successfully:]
[2023-11-27 13:58:15,856]: INFO: common: created directory at: artifacts:]
[2023-11-27 13:58:15,859]: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir:]
[2023-11-27 13:58:15,861]: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir:]
